### Setting paths and imports

In [1]:
import sys
import os
import pandas as pd

module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)
module_path

'/NFSHOME/mprado/AIIM/GRETEL/main2/GRETEL'

In [2]:
os.chdir(module_path)

In [3]:
# from src.evaluation.evaluator_manager import EvaluatorManager
# from src.evaluation.future.evaluator_manager_do import EvaluatorManager as PairedEvaluatorManager
from src.utils.context import Context
from src.data_analysis.future.data_analyzer import DataAnalyzer as data_analyzer

from src.evaluation.future.evaluator_manager_triplets import EvaluatorManager

In [4]:
# config_f_name = 'ensembles/triplets/TCR-TCO/E10-TCR-128-28-0.25_TCO_Ensemble-Bidirectional.jsonc'
config_f_name = 'ensembles/triplets/TCR-TCO/E10-TCR-128-28-0.25_TCO_dl-explainer-selector.jsonc'
# config_f_name = 'ensembles/triplets/TCR-TCO/E10-TCR-128-28-0.25_TCO_RSGG.jsonc'

### Evaluating the explainer

In [5]:
config_path = os.path.join(module_path, 'lab', 'config', config_f_name)
runno = 1
    
print(f"Generating context for: {config_path}")
context = Context.get_context(config_path)
context.run_number = runno

context.logger.info(f"Executing: {context.config_file} Run: {context.run_number}")
context.logger.info("Creating the evaluation manager....................................")

context.logger.info("Creating the evaluators......................................................")
eval_manager = EvaluatorManager(context)

context.logger.info(
    "Evaluating the explainers............................................................."
)
eval_manager.evaluate()

Generating context for: /NFSHOME/mprado/AIIM/GRETEL/main2/GRETEL/lab/config/ensembles/triplets/TCR-TCO/E10-TCR-128-28-0.25_TCO_dl-explainer-selector.jsonc
2024-07-31 18:43:34,162328987 | INFO | 3210177 - Executing: /NFSHOME/mprado/AIIM/GRETEL/main2/GRETEL/lab/config/ensembles/triplets/TCR-TCO/E10-TCR-128-28-0.25_TCO_dl-explainer-selector.jsonc Run: 1
2024-07-31 18:43:34,162328999 | INFO | 3210177 - Creating the evaluation manager....................................
2024-07-31 18:43:34,162329011 | INFO | 3210177 - Creating the evaluators......................................................
2024-07-31 18:43:34,162329064 | INFO | 3210177 - Loading: TreeCyclesRand-266bb3721206805597afc8c4184fe0ab
2024-07-31 18:43:34,162329100 | INFO | 3210177 - Created: TreeCyclesRand-266bb3721206805597afc8c4184fe0ab
2024-07-31 18:43:34,162329112 | INFO | 3210177 - Instantiating: src.oracle.nn.gcn.DownstreamGCN
2024-07-31 18:43:34,162329126 | INFO | 3210177 - Instantiating: torch.optim.RMSprop
2024-07-31 

/NFSHOME/mprado/AIIM/GRETEL/main2/GRETEL/src/explainer/future/ensemble/aggregators/multi_criteria/algorithm.py:11: RuntimeWarning: invalid value encountered in true_divide
  normalized_matrix = (criteria_matrix - min_vals) / (max_vals - min_vals)


2024-07-31 18:45:08,162422993 | INFO | 3210177 - Instantiating: src.evaluation.future.stages.fidelity.Fidelity
2024-07-31 18:45:08,162423013 | INFO | 3210177 - Instantiating: src.evaluation.future.stages.instability.Instability
2024-07-31 18:45:08,162423035 | INFO | 3210177 - Instantiating: src.utils.metrics.ged.GraphEditDistanceMetric
2024-07-31 18:45:08,162423036 | INFO | 3210177 - Instantiating: src.evaluation.future.stages.runtime.Runtime
2024-07-31 18:45:08,162423055 | INFO | 3210177 - Instantiating: src.evaluation.future.stages.ged.GraphEditDistance
2024-07-31 18:45:08,162423067 | INFO | 3210177 - Instantiating: src.evaluation.future.stages.correctness.Correctness
2024-07-31 18:45:08,162423080 | INFO | 3210177 - Instantiating: src.evaluation.future.stages.oracle_calls.OracleCalls
2024-07-31 18:45:08,162423094 | INFO | 3210177 - Instantiating: src.evaluation.future.stages.oracle_accuracy.OracleAccuracy
2024-07-31 18:45:08,162423106 | INFO | 3210177 - Instantiating: src.evaluation.

### Aggregating the stats

In [6]:
results_path = os.path.join(module_path, 'lab', 'output', 'results')
stats_file_path = os.path.join(module_path, 'lab', 'stats', 'results.csv')
res = data_analyzer.create_aggregated_dataframe(results_path)
res.to_csv(stats_file_path)
res

,scope,dataset,oracle,explainer,Runtime,Runtime-std,GraphEditDistance,GraphEditDistance-std,Correctness,Correctness-std,OracleCalls,OracleCalls-std,OracleAccuracy,OracleAccuracy-std,Sparsity,Sparsity-std,Fidelity,Fidelity-std,Instability,Instability-std
0,ensembles,TreeCyclesRand-266bb3721206805597afc8c4184fe0ab,OracleTorch-f6871fb4d33e4a346a85becc90cf4cc1,ExplanationBidirectionalSearch,2.606032,0.00000,1.835938,0.000000,1.000000,0.000000,885.882812,0.000000,1.000000,0.0,0.043744,0.000000,1.000000,0.000000,1.007136,0.000000
1,ensembles,TreeCyclesRand-266bb3721206805597afc8c4184fe0ab,OracleTorch-f6871fb4d33e4a346a85becc90cf4cc1,DatasetLevelExplainerSelector,0.046898,0.00000,4.000000,0.000000,0.203125,0.000000,34.734375,0.000000,0.984375,0.0,0.095293,0.000000,0.171875,0.000000,1.009720,0.000000
2,ensembles,TreeCyclesRand-266bb3721206805597afc8c4184fe0ab,TreeCyclesOracle-8f8e216deff127e0218ff13e1490c1a2,ExplanationBidirectionalSearch,4.657705,4.08938,1.201925,0.009012,0.996094,0.003906,754.437500,148.648438,1.000000,0.0,0.028635,0.000224,0.996094,0.003906,1.019278,0.001071


### Visualizing individual instances

In [ ]:
evaluator = eval_manager.evaluators[0]
evaluator

In [ ]:
exp = evaluator.explanations[2]
og_inst = exp.input_instance
cf_inst = exp.counterfactual_instances[0]

In [ ]:

changes = data_analyzer.get_cf_changes(og_inst, cf_inst, False)

In [ ]:
common_edges = changes['common edges']
added_edges = changes['added edges']
removed_edges = changes['removed edges']
common_nodes = changes['common nodes']
added_nodes = changes['added nodes']
removed_nodes = changes['removed nodes']

print(f'added edges: {added_edges}')
print(f'removed_edges: {removed_edges}')
print(f'added nodes: {added_nodes}')
print(f'removed nodes: {removed_nodes}')

In [ ]:
pos = data_analyzer.draw_graph(og_inst)

In [ ]:
data_analyzer.draw_counterfactual_actions(og_inst, cf_inst, position=pos)